# List of Lists - WILDCARD Usage
Initial Model was a list of Key:Value pairs where Value was a Map:  [AAIDx:{}, AAIDy:{} , AAIDz:{}, ...]

Given AAIDy, find index position in the List. (e.g. it is 1 in above example)

Cannot do WILDCARD search only on List elements, so we need a List of Lists

Modify data model as: [ [AAIDx, AAIDx:{ ...}], [AAIDy, AAIDy:{ ... }], [AAIDz, AAIDz:{....} ] ] 

The following code will fetch the intended index and value.


### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed. 



In [24]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.2.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;
import com.aerospike.client.cdt.ListReturnType;

import java.util.List;
import java.util.HashMap;
import java.util.Map;
import java.util.Random;
import java.util.ArrayList;
import java.util.Arrays;

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);

In [5]:
Key myKey = new Key("test", "testset", "BucketId");
String binName = "myListBin1";
ListPolicy lPolicy = new ListPolicy(ListOrder.UNORDERED, ListWriteFlags.DEFAULT);
client.delete(null,myKey);

false

In [25]:
// Initial Model was a list of Key:Value pairs where Value was a Map:  [AAIDx:{}, AAIDy:{} , AAIDz:{}, ...]
// Given AAIDy, find index position in the List. (e.g. it is 1 in above example)
// Cannot do WILDCARD search only on List elements, so we need a List of Lists
// Modify data model as: [ [AAIDx, AAIDx:{ ...}], [AAIDy, AAIDy:{ ... }], [AAIDz, AAIDz:{....} ] ] 
// The following code will fetch the intended index and value.

//Add record data
List<Value> aaidlist1 = new ArrayList<Value>();
aaidlist1.add(Value.get("AAID1"));
HashMap <String, Integer> mapAAID1 = new HashMap <String, Integer>();
mapAAID1.put("1234", 1800);
mapAAID1.put("4374", 1600);
mapAAID1.put("6876", 1600);
mapAAID1.put("inner_ttl", 1800);
HashMap <String, HashMap> mapAAID_1 = new HashMap <String, HashMap>();
mapAAID_1.put("AAID1", mapAAID1);
aaidlist1.add(Value.get(mapAAID_1));

List<Value> aaidlist2 = new ArrayList<Value>();
aaidlist2.add(Value.get("AAID2"));
HashMap <String, Integer> mapAAID2 = new HashMap <String, Integer>();
mapAAID2.put("7657", 900);
mapAAID2.put("5435", 500);
mapAAID2.put("inner_ttl", 900);
HashMap <String, HashMap> mapAAID_2 = new HashMap <String, HashMap>();
mapAAID_2.put("AAID2", mapAAID2);
aaidlist2.add(Value.get(mapAAID_2));

List<Value> aaidlist3 = new ArrayList<Value>();
aaidlist3.add(Value.get("AAID3"));
HashMap <String, Integer> mapAAID3 = new HashMap <String, Integer>();
mapAAID3.put("9878", 12000);
mapAAID3.put("4454", 1000);
mapAAID3.put("inner_ttl", 12000);
HashMap <String, HashMap> mapAAID_3 = new HashMap <String, HashMap>();
mapAAID_3.put("AAID3", mapAAID3);
aaidlist3.add(Value.get(mapAAID_3));

client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(aaidlist1)));
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(aaidlist2)));
client.operate(null, myKey, ListOperation.append(lPolicy, binName, Value.get(aaidlist3)));

System.out.println("My List after mapInsert: "+ client.get(null, myKey));

My List after mapInsert: (gen:3),(exp:452752299),(bins:(myListBin1:[[AAID1, {AAID1={6876=1600, 4374=1600, 1234=1800, inner_ttl=1800}}], [AAID2, {AAID2={7657=900, 5435=500, inner_ttl=900}}], [AAID3, {AAID3={9878=12000, inner_ttl=12000, 4454=1000}}]]))


In [26]:
List<Value> aaidList = new ArrayList<Value>();
aaidList.add(Value.get("AAID2"));
aaidList.add(Value.get(Value.WILDCARD));

System.out.println(client.operate(null, myKey, ListOperation.getByValue("myListBin1", Value.ListValue.get(aaidList), ListReturnType.INDEX)));

System.out.println(client.operate(null, myKey, ListOperation.getByValue("myListBin1", Value.ListValue.get(aaidList), ListReturnType.VALUE)));

(gen:3),(exp:452752299),(bins:(myListBin1:[1]))
(gen:3),(exp:452752299),(bins:(myListBin1:[[AAID2, {AAID2={7657=900, 5435=500, inner_ttl=900}}]]))


# Cleanup

In [ ]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"